<a href="https://colab.research.google.com/github/Ibrahima07/etl_code/blob/main/elt_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2024-06-28 20:02:10--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip’

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2024-06-28 20:02:11 (864 MB/s) - ‘source.zip’ saved [2707/2707]



In [2]:
!unzip source.zip

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


# **Import module**

In [3]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

### Constante

In [4]:
log_file = "log_file.txt"
target_file = "transformed_data.csv"

# **Extraction data**

### *Extraction data in csv file*

In [5]:
def extract_from_csv(file_to_process):
  dateframe = pd.read_csv(file_to_process)
  return dateframe

### *Extraction data in json file*

In [6]:
def extract_from_json(file_to_process):
  dataframe = pd.read_json(file_to_process, lines=True)
  return dataframe

### *Extraction data in xml file*

In [14]:
def extract_from_xml(file_to_process):
  dataframe = pd.DataFrame(columns=["name", "height", "weight"])
  tree  = ET.parse(file_to_process)
  root = tree.getroot()
  for person in root:
    name = person.find("name").text
    height = float(person.find("height").text)
    weight = float(person.find("weight").text)
    dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True)

  return dataframe


### *Extraction data in all file*

In [16]:
def extract():
  extracted_data = pd.DataFrame(columns=["name", "height", "weight"])

  # process all csv files
  for csvfile in glob.glob("*.csv"):
    extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)

  # process all json files
  for jsonfile in glob.glob("*.json"):
    extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True)

  # process all xml files
  for xmlfile in glob.glob("*.xml"):
    extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True)

  return extracted_data


# **Transformation**

In [17]:
def transform(data):

  '''Convert inches to meters and round off to two decimals
    1 inch is 0.0254 meters '''
  data['height'] = round(data.height * 0.0254,2)

  '''Convert pounds to kilograms and round off to two decimals
    1 pound is 0.45359237 kilograms '''
  data['weight'] = round(data.weight * 0.45359237,2)

  return data

# **Loading**

In [19]:
def load_data(targetfile, data_to_load):
  data_to_load.to_csv(targetfile)

# **Logging**

In [18]:
def log_process(message):
  timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
  now = datetime.now() # get current timestamp
  timestamp = now.strftime(timestamp_format)
  with open(log_file, "a") as f:
    f.write(timestamp + ',' + message + '\n')

# **Testing ETL operations and log progress**

In [20]:
# Log the initialization of the ETL process
log_process("ETL Job Started")

# Log the beginning of the Extraction process
log_process("Extract phase Started")
extracted_data = extract()

# Log the completion of the Extraction process
log_process("Extract phase Ended")

# Log the beginning of the Transformation process
log_process("Transform phase Started")
transformed_data = transform(extracted_data)
print("Transformed Data")
print(transformed_data)

# Log the completion of the Transformation process
log_process("Transform phase Ended")

# Log the beginning of the Loading process
log_process("Load phase Started")
load_data(target_file,transformed_data)

# Log the completion of the Loading process
log_process("Load phase Ended")

# Log the completion of the ETL process
log_process("ETL Job Ended")

Transformed Data
     name  height  weight
0    alex    1.67   51.25
1    ajay    1.82   61.91
2   alice    1.76   69.41
3    ravi    1.73   64.56
4     joe    1.72   65.45
5    alex    1.67   51.25
6    ajay    1.82   61.91
7   alice    1.76   69.41
8    ravi    1.73   64.56
9     joe    1.72   65.45
10   alex    1.67   51.25
11   ajay    1.82   61.91
12  alice    1.76   69.41
13   ravi    1.73   64.56
14    joe    1.72   65.45
15   jack    1.74   55.93
16    tom    1.77   64.18
17  tracy    1.78   61.90
18   john    1.72   50.97
19   jack    1.74   55.93
20    tom    1.77   64.18
21  tracy    1.78   61.90
22   john    1.72   50.97
23   jack    1.74   55.93
24    tom    1.77   64.18
25  tracy    1.78   61.90
26   john    1.72   50.97
27  simon    1.72   50.97
28  jacob    1.70   54.73
29  cindy    1.69   57.81
30   ivan    1.72   51.77
31  simon    1.72   50.97
32  jacob    1.70   54.73
33  cindy    1.69   57.81
34   ivan    1.72   51.77
35  simon    1.72   50.97
36  jacob    1.70   5